In [3]:
import pandas as pd
import yfinance as yf
from datetime import datetime

import requests
from bs4 import BeautifulSoup

# 1. Download SBI Securities List

- Webpage: https://search.sbisec.co.jp/v2/popwin/info/stock/pop6040_etf.html

In [38]:
# URL of the webpage you want to scrape
url = 'https://search.sbisec.co.jp/v2/popwin/info/stock/pop6040_etf.html'

# Send a GET request to the webpage
response = requests.get(url)

# Ensure the request was successful
if response.status_code == 200:
    # Parse the content of the request with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

else:
    print(f"Failed to retrieve the webpage: Status code {response.status_code}")


In [39]:
# Initialize an empty list to hold the results
results = []

# Extract and append text from specific <p> tags with class "fm01"
for p in soup.find_all('p', class_='fm01'):
    # Check if the <p> tag contains an <a> tag, then extract the text and 'href' attribute
    a_tag = p.find('a', href=True)
    if a_tag:
        # Append both the text of the <a> tag and the 'href' attribute to the results
        results.append(a_tag.text)  # This will capture "AAPD"
        results.append(a_tag['href'])  # This captures the URL
    else:
        # If there's no <a> tag, just get the text from the <p> tag
        results.append(p.get_text(strip=True))

# Initialize an empty dict to separate the results
etf_dict = {
    "ticker": [],
    "sbi_link": [],
    "name": [],
    "abstract": [],
    "market": [],
    "expense_ratio": [],
    "factsheet_link": [],
    "issuer": [],
    "nisa_invest_flag": [],
}

# Mapping index to dictionary key
keys = list(etf_dict.keys())

# Iterate over the list with a step of 10
for i in range(0, len(results), 10):
    if results[i] != "銘柄コード":
        # Distribute each item to the corresponding key in the dictionary
        etf_dict["ticker"].append(results[i])
        etf_dict["sbi_link"].append(results[i+1])
        etf_dict["name"].append(results[i+2])
        etf_dict["abstract"].append(results[i+3])
        etf_dict["market"].append(results[i+4])
        etf_dict["expense_ratio"].append(results[i+5])
        etf_dict["factsheet_link"].append(results[i+7])
        etf_dict["issuer"].append(results[i+8])
        etf_dict["nisa_invest_flag"].append(results[i+9])
    else:
        break

In [40]:
df = pd.DataFrame(etf_dict)
df

,ticker,sbi_link,name,abstract,market,expense_ratio,factsheet_link,issuer,nisa_invest_flag
0,AAPD,https://www.sbisec.co.jp/ETGate/?OutSide=on&_C...,Direxion デイリー AAPL 株 ベア1倍 ETF,アップル・インク (NASDAQ:\r\nAAPL)の普通株式の日々のパフォーマンスのインバ...,NASDAQ,0.95%,https://www.direxion.com/uploads/AAPU-AAPD-Fac...,Direxion Investments,
1,AAPU,https://www.sbisec.co.jp/ETGate/?OutSide=on&_C...,Direxion デイリー AAPL 株 ブル 1.5 倍 ETF,アップル・インク (NASDAQ: AAPL)\r\nの普通株式の日々のパフォーマンスの15...,NASDAQ,0.95%,https://www.direxion.com/uploads/AAPU-AAPD-Fac...,Direxion Investments,
2,ACWI,https://www.sbisec.co.jp/ETGate/?OutSide=on&_C...,iシェアーズ MSCI ACWI ETF,MSCI All Country World Indexの価格および利回りの実績に概ね対応す...,NASDAQ,0.32%,https://www.ishares.com/us/literature/fact-she...,ブラックロック･ファンド・アドバイザーズ,〇
3,AFK,https://www.sbisec.co.jp/ETGate/?OutSide=on&_C...,ヴァンエックベクトル アフリカ インデックス ETF,アフリカ・インデックスETFは、ヴァンエック・ベクトルGDPアフリカ指数の価格と利回り（手数...,NYSE Arca,0.98%,https://www.vaneck.com/us/en/investments/afric...,ヴァン・エック・グローバル,〇
4,AGG,https://www.sbisec.co.jp/ETGate/?OutSide=on&_C...,iシェアーズ・コア 米国総合債券市場 ETF,ブルームバーグ・バークレイズ\r\n米国総合インデックスに連動している。米国の投資適格債券市...,NYSE Arca,0.03%,https://www.ishares.com/us/literature/fact-she...,ブラックロック･ファンド・アドバイザーズ,
...,...,...,...,...,...,...,...,...,...
400,XYLD,https://www.sbisec.co.jp/ETGate/?OutSide=on&_C...,グローバルＸ SP 500 カバード コール ETF,CBOE S&P 500 バイライト・インデックスに連動する運用実績を目指す。,NYSE Arca,0.60%,https://www.globalxetfs.com/content/files/XYLD...,グローバルエックス・マネジメント,
401,XYLG,https://www.sbisec.co.jp/ETGate/?OutSide=on&_C...,グローバルＸ SP 500 カバードコール50 ETF,CBOE S&P 500 ハーフ・バイライト・インデックスに連動する運用実績を目指す。,NYSE Arca,0.60%,https://www.globalxetfs.com/content/files/XYLG...,グローバルエックス・マネジメント,
402,YANG,https://www.sbisec.co.jp/ETGate/?OutSide=on&_C...,Direxion デイリー FTSE中国株 ベア 3倍 ETF,FTSEChina50Indexの運用実績（手数料・費用控除前）の逆数の3倍（300％）に連...,NYSE Arca,1.00%,https://www.direxion.com/uploads/YINN-YANG-Fac...,Direxion Investments,
403,YINN,https://www.sbisec.co.jp/ETGate/?OutSide=on&_C...,Direxion デイリー FTSE中国株 ブル 3倍 ETF,FTSE中国50インデックスの運用実績（手数料・費用控除前）の3倍（300％）の日次投資成果...,NYSE Arca,1.42%,https://www.direxion.com/uploads/YINN-YANG-Fac...,Direxion Investments,


In [43]:
df.to_csv("../data/SBI_etf_us_list.csv", index=False)

# 2. Download ETF Data

In [44]:
# Getting the current date
current_date = datetime.now().strftime("%Y-%m-%d")
current_date

'2024-03-05'

In [46]:
data_path = "../data/SBI_etf_us_list.csv"
save_path = "../data/etf_data/price"

In [47]:
etf_df = pd.read_csv(data_path)
etf_df.head(2)

,ticker,sbi_link,name,abstract,market,expense_ratio,factsheet_link,issuer,nisa_invest_flag
0,AAPD,https://www.sbisec.co.jp/ETGate/?OutSide=on&_C...,Direxion デイリー AAPL 株 ベア1倍 ETF,アップル・インク (NASDAQ:\r\nAAPL)の普通株式の日々のパフォーマンスのインバ...,NASDAQ,0.95%,https://www.direxion.com/uploads/AAPU-AAPD-Fac...,Direxion Investments,NaN
1,AAPU,https://www.sbisec.co.jp/ETGate/?OutSide=on&_C...,Direxion デイリー AAPL 株 ブル 1.5 倍 ETF,アップル・インク (NASDAQ: AAPL)\r\nの普通株式の日々のパフォーマンスの15...,NASDAQ,0.95%,https://www.direxion.com/uploads/AAPU-AAPD-Fac...,Direxion Investments,NaN


In [48]:
# start date
start_date = '1900-1-1'

# period
period_internal = '1d'

## 2.1 Price & Dividend

- yfinance

In [49]:
def down_data(ticker, start_date, period_internal, save_path, current_date):
    # Get data on this ticker
    ticker_data = yf.Ticker(ticker)
    
    # Get the historical prices for this ticker
    ticker_df = ticker_data.history(period=period_internal, start=start_date)
    
    # ticker request failed
    if ticker_df.shape[0] != 0:
        # save
        save_file = f"{save_path}/{ticker}_{current_date}.csv"

        ticker_df.to_csv(save_file)

        print(f"{ticker} download & save succeed.")

In [50]:
for ticker in etf_df["ticker"]:
    down_data(ticker, start_date, period_internal, save_path, current_date)

AAPD download & save succeed.
AAPU download & save succeed.
ACWI download & save succeed.
AFK download & save succeed.
AGG download & save succeed.
AGGY download & save succeed.
AIQ download & save succeed.
AIRR download & save succeed.
ALTY download & save succeed.
AMZD download & save succeed.
AMZU download & save succeed.
BBH download & save succeed.
BIV download & save succeed.
BKCH download & save succeed.
BLV download & save succeed.
BND download & save succeed.
BNDX download & save succeed.
BOTZ download & save succeed.
BRF download & save succeed.
BRZU download & save succeed.
BSV download & save succeed.
BUG download & save succeed.
BWX download & save succeed.
CHAU download & save succeed.
CIBR download & save succeed.
CLOU download & save succeed.
CLIP download & save succeed.
CNRG download & save succeed.
CNXT download & save succeed.
COM download & save succeed.
COPX download & save succeed.
CTEC download & save succeed.
CURE download & save succeed.
CWB download & save su

## 2.2 Profile

- etfdb.com

## 2.3 Holdings

- etfdb.com

## 2.4 Region & Country & Sector & Market Cap

- etfdb.com

## 2.5 Technicals

- etfdb.com

## 2.6 Factsheet

# 3. Comparison

# 4. Visualization